# Lógica proposicional
Material interessante que pode ser adaptado https://www.youtube.com/watch?v=D9pkdnIrjdM

## Código em inglês

In [ ]:
# quantifier: "there exists a unique ..."
# e.g. exactly_one( n*n == 9 for n in range(1,10) ) is True
def exactly_one(S):
	return len([ s for s in S if s ]) == 1
#

def in_order(L):
	return all( L[i] <= L[i+1] for i in range(0, len(L)-1) )
#

def set_to_predicate(S):
	return lambda x: x in S
#

def truth_table_rows(variables):
	if len(variables) == 0:
		return [dict()]
	variables = list(variables)
	P = variables[0]
	R = truth_table_rows(variables[1:])
	add_P = lambda v: [ dict([(P,v)] + list(r.items())) for r in R ]
	return add_P(True) + add_P(False)
#

def vars(*var_names):
        return ( Variable(name) for name in var_names )
#

def cast_to_proposition(p):
	if isinstance(p, Proposition):
		return p
	elif isinstance(p, str):
		return Variable(p)
	elif isinstance(p, bool):
		return Constant(p)
	else:
		raise ValueError()
#

class Proposition:
	symbol = ''
	empty_str = ''
	def __init__(self, *children):
		self.children = [ cast_to_proposition(c) for c in children ]
	def __str__(self):
		if len(self.children) == 0: return self.empty_str
		return self.symbol.join( c.child_str() for c in self.children )
	def evaluate(self, **assignments):
		raise NotImplementedError()
	def variables(self):
		if len(self.children) == 0:
			return frozenset()
		else:
			return frozenset.union(*[ c.variables() for c in self.children ])
	def __repr__(self):
		return 'Proposition( {0} )'.format(self)
	def child_str(self):
		return ('{0}' if isinstance(self, (Constant,Variable,Not)) else '({0})').format(self)
	def print_truth_table(self):
		vars = sorted( self.variables() )
		rows = truth_table_rows(vars)
		
		formula_header = str(self)
		formula_len = max(5,len(formula_header))
		header = '{0}  #  {1: ^{2}}'.format('  '.join('{0: ^5}'.format(v) for v in vars), formula_header, formula_len)
		print(header)
		print('#'*len(header))
		
		for r in rows:
			var_cols = '  '.join('{0: ^{1}}'.format(str(r[v]), max(5,len(v))) for v in vars)
			result_col = '{0: ^{1}}'.format(str(self.evaluate(**r)), formula_len)
			print('{0}  #  {1}'.format(var_cols, result_col)) 
		print()
	def to_tree(self):
		from trees import ListTree
		result = ListTree(value=str(self))
		for c in self.children:
			result.add_child_node(c.to_tree())
		return result
	def __and__(self,other):
		v = self.children if isinstance(self,And) else [self]
		w = other.children if isinstance(other,And) else [other]
		return And(*(v+w))
	def __rand__(self,other):
		return cast_to_proposition(other) & self
	def __or__(self,other):
		v = self.children if isinstance(self,Or) else [self]
		w = other.children if isinstance(other,Or) else [other]
		return Or(*(v+w))
	def __ror__(self,other):
		return cast_to_proposition(other) | self
	def __invert__(self):
		return Not(self)
	def __rshift__(self,other):
		return Implies(self,other)
	def __rrshift__(self,other):
		return Implies(other,self)
	def __lshift__(self,other):
		return ImpliedBy(self,other)
	def __rlshift__(self,other):
		return ImpliedBy(other,self)
	def disjunction(self,other):
		return self | other
	def conjunction(self,other):
		return self & other
	def negation(self):
		return ~self
	def implies(self,other):
		return self >> other
	def impliedby(self,other):
		return self << other
	def iff(self,other):
		return Iff(self,other)
	def is_tautology(self):
		return all( self.evaluate(**r) for r in truth_table_rows(self.variables()) )
	def is_contradiction(self):
		return all( not self.evaluate(**r) for r in truth_table_rows(self.variables()) )
	def is_contingency(self):
		return not self.is_tautology() and not self.is_contradiction()
	def __eq__(self,other):
		return self.is_equivalent(other)
	def is_equivalent(self,other):
		other = cast_to_proposition(other)
		return all( self.evaluate(**r) == other.evaluate(**r) for r in truth_table_rows(self.variables() | other.variables()) )
	def is_identical(self,other):
		return self.__class__ == other.__class__ \
			and len(self.children) == len(other.children) \
			and all( c.is_identical(d) for (c,d) in zip(self.children,other.children) )
	def substitute(self, e1, e2):
		if self.is_identical(e1):
			return e2
		else:
			return self.__class__( *[c.substitute(e1,e2) for c in self.children] )
#

class Constant(Proposition):
	def __init__(self,value):
		self.children = []
		self.value = bool(value)
	def substitute(self, e1, e2):
		return Constant(self.value)
	def __str__(self):
		return str(self.value)
	def evaluate(self, **assignments):
		return self.value
	def is_identical(self,other):
		return isinstance(other, Constant) and self.value == other.value
#

class Variable(Proposition):
	def __init__(self,name):
		self.children = []
		self.name = name
	def substitute(self, e1, e2):
		if self.is_identical(e1):
			return e2
		else:
			return Variable(self.name)
	def variables(self):
		return frozenset({ self.name })
	def __str__(self):
		return self.name
	def evaluate(self, **assignments):
		return assignments[self.name]
	def is_identical(self,other):
		return isinstance(other, Variable) and self.name == other.name
#

class Not(Proposition):
	def __init__(self,child):
		Proposition.__init__(self,child)
	def __str__(self):
		return u'¬{0}'.format(self.children[0].child_str()) 
	def evaluate(self, **assignments):
		return not self.children[0].evaluate(**assignments)
#

class And(Proposition):
	symbol = ' ^ '
	empty_str = 'True'
	def evaluate(self, **assignments):
		return all( child.evaluate(**assignments) for child in self.children )
#

class Or(Proposition):
	symbol = ' v '
	empty_str = 'False'
	def evaluate(self, **assignments):
		return any( child.evaluate(**assignments) for child in self.children )
#

class Implies(Proposition):
	symbol = ' => '
	def __init__(self,child1,child2):
		Proposition.__init__(self,child1,child2)
	def evaluate(self, **assignments):
		if self.children[0].evaluate(**assignments):
			return self.children[1].evaluate(**assignments)
		else:
			return True
#

class ImpliedBy(Proposition):
	symbol = ' <= '
	def __init__(self,child1,child2):
		Proposition.__init__(self,child1,child2)
	def evaluate(self, **assignments):
		if self.children[1].evaluate(**assignments):
			return self.children[0].evaluate(**assignments)
		else:
			return True
#

class Iff(Proposition):
	symbol = ' <=> '
	def __init__(self,child1,child2):
		Proposition.__init__(self,child1,child2)
	def evaluate(self, **assignments):
		return self.children[0].evaluate(**assignments) == self.children[1].evaluate(**assignments)
#

class ArgumentForm:
	def __init__(self, *premises, conclusion):
		self.premises = [ cast_to_proposition(c) for c in premises ]
		self.conclusion = cast_to_proposition(conclusion)
	def variables(self):
		return frozenset.union(self.conclusion.variables(), *[ c.variables() for c in self.premises ])
	def __repr__(self):
		return 'ArgumentForm( {0} )'.format(self)
	def __str__(self):
		return ((', '.join(str(c) for c in self.premises) + ', ') if self.premises else '') + 'conclusion = ' + str(self.conclusion)
	def print_truth_table(self):
		vars = sorted( self.variables() )
		rows = truth_table_rows(vars)
		
		var_strings = [ '{0: ^5}'.format(v) for v in vars ]
		premise_strings = [ '{0: ^6}'.format(str(c)) for c in self.premises ]
		conclusion_string = '{0: ^10}'.format(str(self.conclusion))
		vars_header = '  '.join(var_strings)
		premises_header = '{0: ^8}'.format('   '.join(premise_strings))
		print('{0}  #  {1: ^{2}}  #  {3: ^{4}}'.format(' '*len(vars_header), 'premises', len(premises_header), 'conclusion', len(conclusion_string)))
		header = '{0}  #  {1: ^8}  #  {2}'.format(vars_header, premises_header, conclusion_string)
		print(header)
		print('#'*len(header))
		
		for r in rows:
			premise_values = [ c.evaluate(**r) for c in self.premises ]
			conclusion_value = self.conclusion.evaluate(**r)
			star = '*' if all( v for v in premise_values ) else ''
			var_cols = '  '.join( '{0: ^{1}}'.format(str(r[v]), len(k)) for (k,v) in zip(var_strings, vars) )
			premise_cols = '   '.join( '{0: ^{1}}'.format(str(v)+star, len(k)) for (k,v) in zip(premise_strings, premise_values) )
			conclusion_col = '{0: ^{1}}'.format(str(conclusion_value)+star, len(conclusion_string))
			print('{0}  #  {1: ^8}  #  {2}'.format(var_cols, premise_cols, conclusion_col))
		print()
	def is_valid(self):
		vars = (frozenset.union(*[ c.variables() for c in self.premises ]) if self.premises else frozenset()) | self.conclusion.variables()
		return all( self.conclusion.evaluate(**r) for r in truth_table_rows(vars) if all( c.evaluate(**r) for c in self.premises ) )
	def substitute(self, e1, e2):
		return ArgumentForm( *[ c.substitute(e1,e2) for c in self.premises ], conclusion = self.conclusion.substitute(e1,e2) )
#

In [ ]:
P, Q, R  = vars('P', 'Q', 'R')

Proposition(P)
Proposition(Q)
Proposition(R)

Proposition( R )

In [ ]:
formula1 = ((P | Q) & ~P) >> Q

In [ ]:
formula1

Proposition( ((P v Q) ^ ¬P) => Q )

In [ ]:
# Tabela verdade
formula1.print_truth_table()

  P      Q    #  ((P v Q) ^ ¬P) => Q
####################################
True   True   #         True        
True   False  #         True        
False  True   #         True        
False  False  #         True        



In [ ]:
# Tautologia
formula1.is_tautology()

True

In [ ]:
formula2 = (P & Q) >> R

In [ ]:
formula3 = P >>(Q >> R)

In [ ]:
formula2.print_truth_table()

  P      Q      R    #  (P ^ Q) => R
####################################
True   True   True   #      True    
True   True   False  #     False    
True   False  True   #      True    
True   False  False  #      True    
False  True   True   #      True    
False  True   False  #      True    
False  False  True   #      True    
False  False  False  #      True    



In [ ]:
formula2 == formula3

True

In [ ]:
P & Q

Proposition( P ^ Q )

In [ ]:
P | Q

Proposition( P v Q )

In [ ]:
P >> Q

Proposition( P => Q )

In [ ]:
P << Q

Proposition( P <= Q )

In [ ]:
P.iff(Q)

Proposition( P <=> Q )

In [ ]:
# Contradição
(P & ~P).is_contradiction()

True

In [ ]:
(P & ~P).is_contingency()

False

## Código em português

In [ ]:
# quantificador: "há apenas um único ..."
# exemplo exatamente_um( n*n == 9 for n in range(1,10) ) is True

def exatamente_um(S):
	return len([ s for s in S if s ]) == 1
#

def in_order(L):
	return all( L[i] <= L[i+1] for i in range(0, len(L)-1) )
#

def seta_predicado(S):
	return lambda x: x in S
#

def tabela_verdade_linhas(variables):
	if len(variables) == 0:
		return [dict()]
	variables = list(variables)
	P = variables[0]
	R = tabela_verdade_linhas(variables[1:])
	add_P = lambda v: [ dict([(P,v)] + list(r.items())) for r in R ]
	return add_P(True) + add_P(False)
#

def vars(*var_names):
        return ( Variavel(name) for name in var_names )
#

def cast_to_proposition(p):
	if isinstance(p, Proposicao):
		return p
	elif isinstance(p, str):
		return Variavel(p)
	elif isinstance(p, bool):
		return Constante(p)
	else:
		raise ValueError()
#

class Proposicao:
	symbol = ''
	empty_str = ''
	def __init__(self, *children):
		self.children = [ cast_to_proposition(c) for c in children ]
	def __str__(self):
		if len(self.children) == 0: return self.empty_str
		return self.symbol.join( c.child_str() for c in self.children )
	def avaliar(self, **assignments):
		raise NotImplementedError()
	def variaveis(self):
		if len(self.children) == 0:
			return frozenset()
		else:
			return frozenset.union(*[ c.variaveis() for c in self.children ])
	def __repr__(self):
		return 'Proposicao( {0} )'.format(self)
	def child_str(self):
		return ('{0}' if isinstance(self, (Constante,Variavel,Not)) else '({0})').format(self)
	def print_tabela_verdade(self):
		vars = sorted( self.variaveis() )
		rows = tabela_verdade_linhas(vars)
		
		formula_header = str(self)
		formula_len = max(5,len(formula_header))
		header = '{0}  #  {1: ^{2}}'.format('  '.join('{0: ^5}'.format(v) for v in vars), formula_header, formula_len)
		print(header)
		print('#'*len(header))
		
		for r in rows:
			var_cols = '  '.join('{0: ^{1}}'.format(str(r[v]), max(5,len(v))) for v in vars)
			result_col = '{0: ^{1}}'.format(str(self.avaliar(**r)), formula_len)
			print('{0}  #  {1}'.format(var_cols, result_col)) 
		print()
	def to_tree(self):
		from trees import ListTree
		result = ListTree(value=str(self))
		for c in self.children:
			result.add_child_node(c.to_tree())
		return result
	def __and__(self,other):
		v = self.children if isinstance(self,And) else [self]
		w = other.children if isinstance(other,And) else [other]
		return And(*(v+w))
	def __rand__(self,other):
		return cast_to_proposition(other) & self
	def __or__(self,other):
		v = self.children if isinstance(self,Or) else [self]
		w = other.children if isinstance(other,Or) else [other]
		return Or(*(v+w))
	def __ror__(self,other):
		return cast_to_proposition(other) | self
	def __invert__(self):
		return Not(self)
	def __rshift__(self,other):
		return Implica(self,other)
	def __rrshift__(self,other):
		return Implica(other,self)
	def __lshift__(self,other):
		return Implicadopor(self,other)
	def __rlshift__(self,other):
		return Implicadopor(other,self)
	def disjuncao(self,other):
		return self | other
	def conjuncao(self,other):
		return self & other
	def negacao(self):
		return ~self
	def implica(self,other):
		return self >> other
	def implicadopor(self,other):
		return self << other
	def iff(self,other):
		return Iff(self,other)
	def e_tautologia(self):
		return all( self.avaliar(**r) for r in tabela_verdade_linhas(self.variaveis()) )
	def e_contradicao(self):
		return all( not self.avaliar(**r) for r in tabela_verdade_linhas(self.variaveis()) )
	def e_contingencia(self):
		return not self.e_tautologia() and not self.e_contradicao()
	def __eq__(self,other):
		return self.e_equivalente(other)
	def e_equivalente(self,other):
		other = cast_to_proposition(other)
		return all( self.avaliar(**r) == other.avaliar(**r) for r in tabela_verdade_linhas(self.variaveis() | other.variaveis()) )
	def e_identico(self,other):
		return self.__class__ == other.__class__ \
			and len(self.children) == len(other.children) \
			and all( c.e_identico(d) for (c,d) in zip(self.children,other.children) )
	def substitui(self, e1, e2):
		if self.e_identico(e1):
			return e2
		else:
			return self.__class__( *[c.substitui(e1,e2) for c in self.children] )
#

class Constante(Proposicao):
	def __init__(self,value):
		self.children = []
		self.value = bool(value)
	def substitui(self, e1, e2):
		return Constante(self.value)
	def __str__(self):
		return str(self.value)
	def avaliar(self, **assignments):
		return self.value
	def e_identico(self,other):
		return isinstance(other, Constante) and self.value == other.value
#

class Variavel(Proposicao):
	def __init__(self,name):
		self.children = []
		self.name = name
	def substitui(self, e1, e2):
		if self.e_identico(e1):
			return e2
		else:
			return Variavel(self.name)
	def variaveis(self):
		return frozenset({ self.name })
	def __str__(self):
		return self.name
	def avaliar(self, **assignments):
		return assignments[self.name]
	def e_identico(self,other):
		return isinstance(other, Variavel) and self.name == other.name
#

class Not(Proposicao):
	def __init__(self,child):
		Proposicao.__init__(self,child)
	def __str__(self):
		return u'¬{0}'.format(self.children[0].child_str()) 
	def avaliar(self, **assignments):
		return not self.children[0].avaliar(**assignments)
#

class And(Proposicao):
	symbol = ' ^ '
	empty_str = 'True'
	def avaliar(self, **assignments):
		return all( child.avaliar(**assignments) for child in self.children )
#

class Or(Proposicao):
	symbol = ' v '
	empty_str = 'False'
	def avaliar(self, **assignments):
		return any( child.avaliar(**assignments) for child in self.children )
#

class Implica(Proposicao):
	symbol = ' => '
	def __init__(self,child1,child2):
		Proposicao.__init__(self,child1,child2)
	def avaliar(self, **assignments):
		if self.children[0].avaliar(**assignments):
			return self.children[1].avaliar(**assignments)
		else:
			return True
#

class Implicadopor(Proposicao):
	symbol = ' <= '
	def __init__(self,child1,child2):
		Proposicao.__init__(self,child1,child2)
	def avaliar(self, **assignments):
		if self.children[1].avaliar(**assignments):
			return self.children[0].avaliar(**assignments)
		else:
			return True
#

class Iff(Proposicao):
	symbol = ' <=> '
	def __init__(self,child1,child2):
		Proposicao.__init__(self,child1,child2)
	def avaliar(self, **assignments):
		return self.children[0].avaliar(**assignments) == self.children[1].avaliar(**assignments)
#

class ArgumentForm:
	def __init__(self, *premises, conclusion):
		self.premises = [ cast_to_proposition(c) for c in premises ]
		self.conclusion = cast_to_proposition(conclusion)
	def variaveis(self):
		return frozenset.union(self.conclusion.variaveis(), *[ c.variaveis() for c in self.premises ])
	def __repr__(self):
		return 'ArgumentForm( {0} )'.format(self)
	def __str__(self):
		return ((', '.join(str(c) for c in self.premises) + ', ') if self.premises else '') + 'conclusion = ' + str(self.conclusion)
	def print_tabela_verdade(self):
		vars = sorted( self.variaveis() )
		rows = tabela_verdade_linhas(vars)
		
		var_strings = [ '{0: ^5}'.format(v) for v in vars ]
		premise_strings = [ '{0: ^6}'.format(str(c)) for c in self.premises ]
		conclusion_string = '{0: ^10}'.format(str(self.conclusion))
		vars_header = '  '.join(var_strings)
		premises_header = '{0: ^8}'.format('   '.join(premise_strings))
		print('{0}  #  {1: ^{2}}  #  {3: ^{4}}'.format(' '*len(vars_header), 'premises', len(premises_header), 'conclusion', len(conclusion_string)))
		header = '{0}  #  {1: ^8}  #  {2}'.format(vars_header, premises_header, conclusion_string)
		print(header)
		print('#'*len(header))
		
		for r in rows:
			premise_values = [ c.avaliar(**r) for c in self.premises ]
			conclusion_value = self.conclusion.avaliar(**r)
			star = '*' if all( v for v in premise_values ) else ''
			var_cols = '  '.join( '{0: ^{1}}'.format(str(r[v]), len(k)) for (k,v) in zip(var_strings, vars) )
			premise_cols = '   '.join( '{0: ^{1}}'.format(str(v)+star, len(k)) for (k,v) in zip(premise_strings, premise_values) )
			conclusion_col = '{0: ^{1}}'.format(str(conclusion_value)+star, len(conclusion_string))
			print('{0}  #  {1: ^8}  #  {2}'.format(var_cols, premise_cols, conclusion_col))
		print()
	def e_valido(self):
		vars = (frozenset.union(*[ c.variaveis() for c in self.premises ]) if self.premises else frozenset()) | self.conclusion.variaveis()
		return all( self.conclusion.avaliar(**r) for r in tabela_verdade_linhas(vars) if all( c.avaliar(**r) for c in self.premises ) )
	def substitui(self, e1, e2):
		return ArgumentForm( *[ c.substitui(e1,e2) for c in self.premises ], conclusion = self.conclusion.substitui(e1,e2) )
#

In [ ]:
# Could not use => or ->, it gets the error "Code is unreachable. Statements must be separated by newlines or semicolons".

In [ ]:
P, Q, R  = vars('P', 'Q', 'R')

Proposicao(P)
Proposicao(Q)
Proposicao(R)

formula1 = ((P | Q) & ~P) >> Q
formula1.print_tabela_verdade()

  P      Q    #  ((P v Q) ^ ¬P) => Q
####################################
True   True   #         True        
True   False  #         True        
False  True   #         True        
False  False  #         True        



In [ ]:
formula1.e_tautologia()

True

In [ ]:
formula2 = (P & Q) >> R
formula3 = P >>(Q >> R)

In [ ]:
formula2.print_tabela_verdade()

  P      Q      R    #  (P ^ Q) => R
####################################
True   True   True   #      True    
True   True   False  #     False    
True   False  True   #      True    
True   False  False  #      True    
False  True   True   #      True    
False  True   False  #      True    
False  False  True   #      True    
False  False  False  #      True    



In [ ]:
formula2 == formula3

True

In [ ]:
P & Q

Proposicao( P ^ Q )

In [ ]:
P | Q

Proposicao( P v Q )

In [ ]:
P >> Q

Proposicao( P => Q )

In [ ]:
P << Q

Proposicao( P <= Q )

In [ ]:
P.iff(Q)

Proposicao( P <=> Q )

In [ ]:
# Contradição
(P & ~P).e_contradicao()

True

In [ ]:
(P & ~P).e_contingencia()

False

# Exercícios

### EXERCÍCIO 1: 

Construa as tabelas verdades das fórmulas ``(p=>q)=>r`` e ``p=>(q=>r)``. Elas são equivalentes?

*ref: gomide página 36 Exercício 3.4*

In [ ]:
P, Q, R  = vars('P', 'Q', 'R')

Proposicao(P)
Proposicao(Q)
Proposicao(R)

formula1 = (P>>Q)>>R
formula2 = P>>(Q>>R)



In [ ]:
formula1.print_tabela_verdade()

  P      Q      R    #  (P => Q) => R
#####################################
True   True   True   #      True     
True   True   False  #      False    
True   False  True   #      True     
True   False  False  #      True     
False  True   True   #      True     
False  True   False  #      False    
False  False  True   #      True     
False  False  False  #      False    



In [ ]:
formula2.print_tabela_verdade()

  P      Q      R    #  P => (Q => R)
#####################################
True   True   True   #      True     
True   True   False  #      False    
True   False  True   #      True     
True   False  False  #      True     
False  True   True   #      True     
False  True   False  #      True     
False  False  True   #      True     
False  False  False  #      True     



In [ ]:
print("As proposições são equivalente?", formula1 == formula2)

As proposições são equivalente? False


### EXERCÍCIO 2: 
Construa a tabela verdade das proposições abaixo e determine se são tautologia, contradição ou nem uma nem outra.

1. ``(p^¬q)<=>(p=>q)``
2. ``¬p <=>p ``
3. ``(p^¬p)=>q``

*ref: Gomide página 39, exercício 3.16*

In [ ]:
P, Q, R  = vars('P', 'Q', 'R')

Proposicao(P)
Proposicao(Q)
Proposicao(R)

#Item 1
formula1 = (P&~Q).iff(P>>Q)

#Item 2 
formula2 = (~P).iff(P)

#Item 3
formula3 = (P&~P)>>Q

In [ ]:
#Item 1
print("Tabela verdade:\n")
formula1.print_tabela_verdade()
print("É tautologia?",formula1.e_tautologia())
print("É contradição?",formula1.e_contradicao())

Tabela verdade:

  P      Q    #  (P ^ ¬Q) <=> (P => Q)
######################################
True   True   #          False        
True   False  #          False        
False  True   #          False        
False  False  #          False        

É tautologia? False
É contradição? True


In [ ]:
#Item 2
print("Tabela verdade:\n")
formula2.print_tabela_verdade()
print("É tautologia?",formula2.e_tautologia())
print("É contradição?",formula2.e_contradicao())

Tabela verdade:

  P    #  ¬P <=> P
##################
True   #   False  
False  #   False  

É tautologia? False
É contradição? True


In [ ]:
#Item 3
print("Tabela verdade:\n")
formula3.print_tabela_verdade()
print("É tautologia?",formula3.e_tautologia())
print("É contradição?",formula3.e_contradicao())

Tabela verdade:

  P      Q    #  (P ^ ¬P) => Q
##############################
True   True   #      True     
True   False  #      True     
False  True   #      True     
False  False  #      True     

É tautologia? True
É contradição? False


### Exercício 3: 

Verifique se as seguintes afirmações são corretas:

1. ``(¬p^(pVq))`` é logicamente equivalente a ``q``.
2. ``p=>(q^r)`` é logicamente equivalente a ``(p=>q)^(p=>r)``.
3. ``((p=>q)=>r)`` é logicamente equivalente a ``(p=>(q=>r))``.

*ref: Gomide página 41, exercício 3.19*

In [ ]:
P, Q, R  = vars('P', 'Q', 'R')

Proposicao(P)
Proposicao(Q)
Proposicao(R)

#Item 1
formula1 = (~P&(P | Q))

#Item 2
formula2 = P>>(Q&R) 
formula3 = (P>>Q)&(P>>R)

#Item 3
formula4 = ((P>>Q)>>R)
formula5 = (P>>(Q>>R))

In [ ]:
#Item 1
print("São equivalentes?", formula1 == Q)

São equivalentes? False


In [ ]:
#Item 2
print("São equivalentes?", formula2 == formula3)

São equivalentes? True


In [ ]:
#Item 3
print("São equivalentes?", formula4 == formula5)

São equivalentes? False
